In [5]:
import pandas as pd

# Load Kaggle dataset
try:
    df_kaggle = pd.read_csv("/train.csv")
except FileNotFoundError:
    print("Error: train.csv not found.")
    exit(1)

# Handle missing/empty values
df_kaggle = df_kaggle.dropna(subset=["comment_text"])  # Remove rows with NaN
df_kaggle = df_kaggle[df_kaggle["comment_text"].str.strip() != ""]  # Remove empty or whitespace-only strings
df_kaggle = df_kaggle[~df_kaggle["comment_text"].isin(["", " "])]  # Explicitly drop "" or " "

# Map Kaggle labels to your categories
df_kaggle["hate_speech"] = df_kaggle["identity_hate"] | df_kaggle["toxic"]
df_kaggle["cyberbullying"] = df_kaggle["insult"] | df_kaggle["toxic"]
df_kaggle["incitement_violence"] = df_kaggle["threat"]
df_kaggle["threat_safety"] = df_kaggle["threat"] | df_kaggle["severe_toxic"]

# Select relevant columns (excluding fake_account for training)
df_kaggle = df_kaggle[["comment_text", "hate_speech", "cyberbullying", "incitement_violence", "threat_safety"]]
df_kaggle.rename(columns={"comment_text": "text"}, inplace=True)

# Subset for faster training
df_kaggle = df_kaggle.sample(n=10000, random_state=42)  # Adjust to 10,000 if needed

# Oversample rare labels
positive_samples = df_kaggle[df_kaggle["threat_safety"] == 1]
df_kaggle = pd.concat([df_kaggle, positive_samples.sample(n=500, replace=True)], ignore_index=True)

# Save as training_data.csv
df_kaggle.to_csv("trained_data.csv", index=False)
print("Processed Kaggle dataset saved to training_data.csv")

Processed Kaggle dataset saved to training_data.csv


In [ ]:
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

class SocialMediaDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.FloatTensor(label)
        }

try:
    df = pd.read_csv("/content/training_data.csv")
except FileNotFoundError:
    print("Error: training_data.csv not found. Run preprocess_data.py first.")
    exit(1)

# Verify columns
label_columns = ["hate_speech", "cyberbullying", "incitement_violence", "threat_safety"]
if not all(col in df.columns for col in label_columns):
    print(f"Error: Missing columns in training_data.csv. Expected: {label_columns}, Found: {list(df.columns)}")
    exit(1)

# Clean data
df = df.dropna(subset=["text"])
df = df[df["text"].str.strip() != ""]
df[label_columns] = df[label_columns].fillna(0)

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
train_texts, val_texts = train_df["text"].values, val_df["text"].values
train_labels, val_labels = train_df[label_columns].values, val_df[label_columns].values

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=4)

train_dataset = SocialMediaDataset(train_texts, train_labels, tokenizer)
val_dataset = SocialMediaDataset(val_texts, val_labels, tokenizer)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = (pred.predictions > 0.5).astype(int)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='micro')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

training_args = TrainingArguments(
    output_dir="./cyber_shield_model",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=50,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()
model.save_pretrained("./cyber_shield_model")
tokenizer.save_pretrained("./cyber_shield_model")
print("Model and tokenizer saved to ./cyber_shield_model")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

...

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sukhmanarora01 (sukhmanarora01-amity-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


# New section